In [4]:
import findspark
findspark.init('/home/ubuntu/spark-2.1.1-bin-hadoop2.7')
import pyspark
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('Integrate_various_data_sources_new_csv').getOrCreate()
df = spark.read.csv('../Bike-Sharing-Dataset-hour_new.csv', header=True, inferSchema=True)

df.createOrReplaceTempView("dataset")
#query the entire data in sequenced numbers and include lag values
df = spark.sql("SELECT *, LAG(cnt,1,0) OVER (ORDER BY dteday, yr, hr) AS cnt_lag1 "
               "FROM dataset ORDER BY dteday, yr, hr ")

In [5]:
#create a new csv with lag1
import csv
x = 1 # counter
with open('../Bike-Sharing-Dataset-hour_lag1.csv','w') as csvfile:
    fieldnames = ['reinstant', 'dteday', 'season', 'yr', 'mnth', 'hr', 'holiday', 'weekday', 'workingday', 
                  'weathersit', 'temp', 'atemp', 'hum', 'windspeed', 'casual', 'registered', 'cnt', 'cnt_lag1']
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
    writer.writeheader()
    
    for row in df.collect(): 
        writer.writerow({'reinstant':x, 'dteday':str(row[1]), 'season':str(row[2]), 'yr':str(row[3]), 'mnth':str(row[4]), 
                         'hr':str(row[5]), 'holiday':str(row[6]), 'weekday':str(row[7]), 'workingday':str(row[8]), 
                         'weathersit':str(row[9]), 'temp':str(row[10]), 'atemp':str(row[11]), 'hum':str(row[12]), 
                         'windspeed':str(row[13]), 'casual':str(row[14]), 'registered':str(row[15]), 'cnt':str(row[16]), 
                         'cnt_lag1':str(row[17])})
        x += 1